In [1]:
!pip install pandas pickle-mixin scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 44.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 47.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 38.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 35.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 52.1 MB/s eta 0:00:00
  Created wheel for pickle-mixin: filename=pickle_mixin-1.0.2-py3-none-any.whl size=5990 sha256=9f66c1320230b2b05ed18f8d457617f77cb2234a41bfcfc81e972fd1a4629d28
  Stored in directory: /home/pylay/.cache/pip/wheels/f7/98/15/9ec9f295ceae898ee4d69a51f6663696a45d68367e8ea86f6f
Successfully built pickle-mixin

[notice] A new release of pip available: 22.3

In [2]:
import pandas as pd
df = pd.read_csv("datatest.csv")
df = df[['Temperature', 'Humidity', 'CO2']]
df.to_csv("dataForTraining.csv")

In [4]:
data = pd.read_csv("dataForTraining.csv")

#Визначення ідеальних значень параметрів мікроклімату
ideal_values = {
    'Temperature': 22,  # Температура
    'Humidity': 50,     # Вологість
    'CO2': 500          # Рівень CO2
}

# Визначення коефіцієнтів масштабування для кожного параметра
scaling_factors = {
    'Temperature': 0.1,
    'Humidity': 0.1,
    'CO2': 0.01
}

#Відхилення
weighted_deviations = {
    'Temperature': ((data['Temperature'] - ideal_values['Temperature']) / ideal_values['Temperature']) ** 2,
    'Humidity': ((data['Humidity'] - ideal_values['Humidity']) / ideal_values['Humidity']) ** 2,
    'CO2': ((data['CO2'] - ideal_values['CO2']) / ideal_values['CO2']) ** 2
}

# Зробимо нормалізацію та збережемо для подальших дій
normalized_weighted_deviations = {
    'Temperature': weighted_deviations['Temperature'] / weighted_deviations['Temperature'].max(),
    'Humidity': weighted_deviations['Humidity'] / weighted_deviations['Humidity'].max(),
    'CO2': weighted_deviations['CO2'] / weighted_deviations['CO2'].max()
}

weighted_sum_deviations = (
    scaling_factors['Temperature'] * normalized_weighted_deviations['Temperature'] +
    scaling_factors['Humidity'] * normalized_weighted_deviations['Humidity'] +
    scaling_factors['CO2'] * normalized_weighted_deviations['CO2']
)

data['Productivity'] = 100 - (weighted_sum_deviations * 100)

#Обмежимо значення продуктивності, зробимо їх в межах від 1 до 100
data['Productivity'] = data['Productivity'].clip(1, 100)

data = data.sort_values(by=['Temperature', 'Humidity', 'CO2'])
data['Productivity'].describe()

#Виберемо всі ці стовпці для подальших дій
data = data[['Temperature', 'Humidity', 'CO2', 'Productivity']]

In [5]:
data.to_csv("dataForTrainingv1.csv", index=False)
#Видалимо рядки з пропущеними значеннями
data_not_empty = data.dropna(axis=0)

#Не забудемо визначити залежність y та незалежні змінні X
y = data.Productivity
X = data[['Temperature', 'Humidity', 'CO2']]

X.describe()

,Temperature,Humidity,CO2
count,2665.000000,2665.000000,2665.000000
mean,21.433876,25.353937,717.906470
std,1.028024,2.436842,292.681718
min,20.200000,22.100000,427.500000
25%,20.650000,23.260000,466.000000
50%,20.890000,25.000000,580.500000
75%,22.356667,26.856667,956.333333
max,24.408333,31.472500,1402.250000


In [10]:
#Оберемо модель регресійних дерев рішень, оскільки вони здатні обробляти нелінійні залежності
from sklearn.tree import DecisionTreeRegressor

productivity_model = DecisionTreeRegressor(random_state=1)
productivity_model.fit(X, y)

DecisionTreeRegressor(random_state=1)

In [12]:
import pickle

filename = 'ProductivityPredictionModel.sav'
pickle.dump(productivity_model, open(filename, 'wb'))